cartesian slices of spherical dataset via custom fields and cut regions?

In [1]:
import yt
import numpy as np 
from yt.testing import fake_amr_ds

In [2]:
ds = fake_amr_ds(geometry='spherical')

yt : [INFO     ] 2023-11-27 10:24:37,477 Parameters: current_time              = 0.0
yt : [INFO     ] 2023-11-27 10:24:37,478 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2023-11-27 10:24:37,479 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-11-27 10:24:37,480 Parameters: domain_right_edge         = [1.         3.14159265 6.28318531]
yt : [INFO     ] 2023-11-27 10:24:37,480 Parameters: cosmological_simulation   = 0


define a plane in cartesian coordinates:

use 2 points on the surface and the sphere center to define 2 vectors.

Define points in spherical coordinates (r, theta, phi):

In [3]:
pt1 = (1.0, 20. * np.pi/180, 45. * np.pi/180)
pt2 = (1.0, 35. * np.pi/180, 75. * np.pi/180)
pt3 = (0., 0., 0.)

In [4]:
def to_cartesian(r, theta, phi):
    z = r * np.cos(theta)
    xy = r * np.sin(theta)
    x = xy * np.cos(phi)
    y = xy * np.sin(phi)
    return x, y, z

pt3_xyz = np.array(to_cartesian(*pt3))

In [5]:
def _dist_from_plane(field, data):
    # always recalculate the cartesian plane in case
    # points have changed
    pt1_xyz = np.array(to_cartesian(*pt1))
    pt2_xyz = np.array(to_cartesian(*pt2))
    vec1 = pt1_xyz - pt3_xyz
    vec2 = pt2_xyz - pt3_xyz
    normal_vec = np.cross(vec1, vec2)
    vec_mag = np.sqrt(np.sum(normal_vec ** 2))
    normal_vec = normal_vec / vec_mag
    
    # get cartesian points of each data point
    r = data['index', 'r'].to('code_length').d
    theta = data['index', 'theta'].d
    phi = data['index', 'phi'].d
    x, y, z = to_cartesian(r, theta, phi)
    
    # vector from each point to point on plane
    dist_vec = (x - pt1_xyz[0], y - pt1_xyz[1], z - pt1_xyz[2])
    
    # distance to plane for each point 
    dist = np.zeros(x.shape)
    for idim in range(3):
        dist += dist_vec[idim] * normal_vec[idim]
    dist = np.abs(dist) 
    return data.ds.arr(dist, 'code_length')

In [6]:
ds.add_field(
    name=("index", "dist_from_plane"),
    function=_dist_from_plane,
    sampling_type="local",
    units="code_length",
    force_override=True,
)

In [7]:
cr = ds.cut_region(ds.all_data(), ["obj[('index', 'dist_from_plane')] < 0.01"])

In [8]:
d = cr[('stream', 'Density')]
r = cr[('index', 'r')]
theta = cr[('index', 'theta')]
phi = cr[('index', 'phi')]

now need to re-sample into 2d image in some manner... first go, project each point into 2d plane, use a pyplot scatter plot, ignore any interesecting samples in direction normal to plane. 

In [10]:
x, y, z = to_cartesian(r, theta, phi)  # raw data positions within cut region